In [ ]:
import requests
import pandas as pd
from io import StringIO
import time
def criar_dicionario(df):
    """
    Cria um dicionário de categorias com base na coluna 'control'.

    Args:
        df (pd.DataFrame): O DataFrame contendo os dados.

    Returns:
        dict: Um dicionário onde as chaves são os prefixos (antes do '_')
              e os valores são as categorias correspondentes.
    """
    categoria_dict = {}
    categoria_atual = None

    for index, row in df.iterrows():
        control = row['control']
        if "_" not in control:
            categoria_atual = row['produto']  # Encontrou uma nova categoria
        else:
            prefixo = control.split("_")[0].lower()
            categoria_dict[prefixo] = categoria_atual

    return categoria_dict

# Criar o dicionário de categorias


def extrair_categorias_hierarquicas(df, categoria_dict):
    """
    Extrai as categorias e subcategorias do DataFrame, utilizando um dicionário de categorias.

    Args:
        df (pd.DataFrame): O DataFrame contendo os dados.
        categoria_dict (dict): Um dicionário mapeando prefixos de subcategorias para categorias.

    Returns:
        pd.DataFrame: O DataFrame com as colunas 'Categoria' e 'Subcategoria' adicionadas.
    """
    categorias = []
    subcategorias = []
    categoria_atual = None

    for index, row in df.iterrows():
        control = row['control']
        produto = row['produto']

        if "_" not in control:
            categoria_atual = produto
            categoria = categoria_atual
            subcategoria = produto
        else:
            prefixo = control.split("_")[0].lower()
            categoria = categoria_dict.get(prefixo, "Outros")
            subcategoria = produto

        categorias.append(categoria)
        subcategorias.append(subcategoria)

    # df['Categoria'] = categorias
    # df['Subcategoria'] = subcategorias
    return categorias, subcategorias


In [ ]:
def _filter_comercializacao(df:pd.DataFrame) -> pd.DataFrame:
    df = df.rename(columns={'cultivar': 'produto'})
    df['control'] = df['control'].fillna(df['produto'])
    for col in df.columns:
        if col.isdigit():
            df[col] = df[col].astype(str).replace('*','0').replace('nd','0').fillna('0').str.replace(',','.').astype(float)
    return df

def _filter_processamento(df:pd.DataFrame) -> pd.DataFrame:
    df = df.rename(columns={'cultivar': 'produto'})
    df['control'] = df['control'].fillna(df['produto'])
    for col in df.columns:
        if col.isdigit():
            df[col] = df[col].astype(str).replace('*','0').replace('nd','0').str.replace('+','0').fillna('0').str.replace(',','.').astype(float)
    return df

In [43]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import pandas as pd
from io import BytesIO
import io

url1 = 'http://vitibrasil.cnpuv.embrapa.br/'  # Substitua pela URL real do arquivo CSV
url2 = 'http://vitibrasil.cnpuv.embrapa.br/download/ExpSuco.csv'  # Substitua pela URL real do arquivo CSV

try:
    session = requests.Session()
    retry_strategy = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    response = session.get(url1, timeout=10, verify=False)
    response.raise_for_status()
    response = session.get(url2, timeout=10, verify=False)
    bio = BytesIO(response.content)
    wrapper = io.TextIOWrapper(bio, encoding='utf-8')
    df = pd.read_csv(wrapper, sep=None, engine='python', encoding='utf-8')
    df.columns = df.columns.str.lower()
    # if 'comercio' in url2.lower():
    #     df = _filter_comercializacao(df)
    # if 'processa' in url2.lower():
    #     df = _filter_processamento(df)
    # if not df.empty:
    #     categoria_dict = criar_dicionario(df)
    #     categorias, subcategorias = extrair_categorias_hierarquicas(df,categoria_dict)
    #     df['categoria'] = categorias
    #     df['produto'] = subcategorias
    #     df = df[df['categoria'] != df['produto']]
    #     df = df.drop(columns=['id', 'control',])
    session.close()
except requests.exceptions.RequestException as e:
    print(f'Ocorreu um erro durante a requisição: {e}')
except Exception as e:
    print(f'Ocorreu um erro inesperado: {e}')


In [44]:
df.head(5)

,id,país,1970,1970.1,1971,1971.1,1972,1972.1,1973,1973.1,...,2020,2020.1,2021,2021.1,2022,2022.1,2023,2023.1,2024,2024.1
0,1,África do Sul,0,0,0,0,0,0,0,0,...,0,0,0,0,34344,49753,0,0,0,0
1,2,"Alemanha, República Democrática da",0,0,0,0,9962,6514,238418,160979,...,86,44,13,4,5,24,33,39,6,17
2,3,Angola,0,0,0,0,0,0,0,0,...,17766,21627,6073,5915,86536,91839,55683,68724,4974,9112
3,4,Antígua e Barbuda,0,0,0,0,0,0,0,0,...,36,25,120,168,48,57,0,0,0,0
4,5,Antilhas Holandesas,0,0,0,0,1125,945,144,135,...,0,0,0,0,0,0,0,0,0,0


In [45]:
def agrupar_colunas_por_ano(df):
    # Identificar colunas numéricas (anos)
    colunas_anos = {}
    
    # Agrupar colunas pelo ano base
    for col in df.columns:
        # Verificar se a coluna começa com um número (potencialmente um ano)
        if isinstance(col, str) and col.split('.')[0].isdigit():
            ano_base = col.split('.')[0]  # Pega a parte antes do ponto
            if ano_base not in colunas_anos:
                colunas_anos[ano_base] = []
            colunas_anos[ano_base].append(col)
        elif isinstance(col, (int, float)) or (isinstance(col, str) and col.isdigit()):
            # Caso a coluna seja diretamente um número ou string numérica
            ano_base = str(int(float(col)))
            if ano_base not in colunas_anos:
                colunas_anos[ano_base] = []
            colunas_anos[ano_base].append(col)
    
    # Criar um novo DataFrame com as colunas não-numéricas
    colunas_nao_numericas = [col for col in df.columns if col not in [c for sublist in colunas_anos.values() for c in sublist]]
    df_novo = df[colunas_nao_numericas].copy()
    
    # Adicionar as colunas de anos somadas
    for ano, colunas in colunas_anos.items():
        # Converter todas as colunas para numérico, tratando valores não numéricos
        for col in colunas:
            if df[col].dtype == 'object':
                df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce').fillna(0)
        
        # Somar as colunas do mesmo ano
        df_novo[ano] = df[colunas].sum(axis=1)
    
    return df_novo

In [46]:
df_agrupado = agrupar_colunas_por_ano(df)

In [47]:
df_agrupado.columns

Index(['id', 'país', '1970', '1971', '1972', '1973', '1974', '1975', '1976',
       '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985',
       '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994',
       '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021',
       '2022', '2023', '2024'],
      dtype='object')

In [48]:
df_pivoted = df_agrupado.melt(id_vars=[col for col in df_agrupado.columns if not col.isdigit()], 
            var_name='ano', 
            value_name='quantidade'
        ).reset_index(drop=False)
df_pivoted.drop(columns=['id'], inplace=True)

In [49]:
df_pivoted

,index,país,ano,quantidade
0,0,África do Sul,1970,0
1,1,"Alemanha, República Democrática da",1970,0
2,2,Angola,1970,0
3,3,Antígua e Barbuda,1970,0
4,4,Antilhas Holandesas,1970,0
...,...,...,...,...
6870,6870,Trinidade e Tobago,2024,0
6871,6871,Turquia,2024,835
6872,6872,Uruguai,2024,0
6873,6873,Vanuatu,2024,0
